In [1]:
import numpy as np
from scipy.signal import argrelextrema
import ccxt
import pandas as pd
import numba as nb
from sklearn.preprocessing import StandardScaler, RobustScaler
import talib
import plotly.graph_objs as go
from plotly.subplots import make_subplots

pd.options.plotting.backend = "plotly"
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
nb.jit
def create_target(df, long, method='polyfit', polyfit_var='close', pct=0.1):
    if method == 'polyfit':

        trend_list = []
        slope_list = []
        start_list = []
        end_list = []

        index_l = np.arange(long)
        rolling_df = df[polyfit_var].rolling(window=long, min_periods=long)
        for roll in rolling_df:
            if len(roll) < long:
                continue
            slope_array = np.round(np.polyfit(index_l, roll.values, deg=1)[-2], decimals=8)
            slope_list.append(slope_array)
            trend_list.append(np.where(slope_array > 0, 1, np.where(slope_array == 0, 0, -1)).tolist())
            start_list.append(roll.index[0])
            end_list.append(roll.index[long-1])

    y = pd.DataFrame({'trend': trend_list, 'slope': slope_list, 'start_windows': start_list, 'end_windows': end_list})
    return y

In [3]:
def adjusted_sigmoid(x, k=0.5, x0=0):
    """
    Adjusted sigmoid function to map values to the range [-1, 1].
    k controls the steepness of the curve.
    x0 is the midpoint of the sigmoid.
    """
    return 2 / (1 + np.exp(-k * (x - x0))) - 1

In [4]:
def download_1000_candles(market='ADA/USDT', tf='1h'):
    ex = ccxt.binance()
    ohlcv = ex.fetch_ohlcv(market, tf, limit=1001)  # Fetch one extra to ensure 1000 closed candles

    # Build DataFrame
    header = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlcv_df = pd.DataFrame(ohlcv, columns=header)
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms', utc=True)
    # ohlcv_df.set_index('timestamp', inplace=True)

    # Remove the last row to ensure all candles are closed
    ohlcv_df = ohlcv_df.iloc[:-1].copy()

    return ohlcv_df

# download_1000_candles()

,timestamp,open,high,low,close,volume
0,2023-12-12 12:00:00+00:00,0.5917,0.5967,0.5875,0.5937,12610887.7
1,2023-12-12 13:00:00+00:00,0.5938,0.6047,0.5923,0.5950,18382394.2
2,2023-12-12 14:00:00+00:00,0.5949,0.5996,0.5824,0.5876,20177873.3
3,2023-12-12 15:00:00+00:00,0.5875,0.5900,0.5774,0.5825,19485243.7
4,2023-12-12 16:00:00+00:00,0.5824,0.5842,0.5720,0.5744,14873313.4
...,...,...,...,...,...,...
994,2024-01-22 22:00:00+00:00,0.4834,0.4860,0.4780,0.4813,5997442.9
995,2024-01-22 23:00:00+00:00,0.4814,0.4833,0.4769,0.4786,7263183.5
996,2024-01-23 00:00:00+00:00,0.4787,0.4826,0.4780,0.4815,5014864.6
997,2024-01-23 01:00:00+00:00,0.4817,0.4832,0.4777,0.4809,3233567.6


In [90]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import talib  # Ensure TA-Lib is installed

def analyze_and_plot_bitcoin_ranges(df, range_kernel=25):
    # Ensure the timestamp is the index
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    # Calculate ATR as a percentage of the close price
    atr_perc = talib.ATR(df['high'], df['low'], df['close'], timeperiod=100) / df['close'] * 100
    df['hlc3'] = (df['high'] + df['low'] + df['close']) / 3

    # Calculate max and min ranges
    df['max_range'] = (df['close'].shift(-range_kernel).rolling(range_kernel).max() / df['close'] - 1) * 100
    df['min_range'] = (df['close'].shift(-range_kernel).rolling(range_kernel).min() / df['close'] - 1) * 100

    df['max_range_atr'] = df['max_range'] / atr_perc
    df['min_range_atr'] = df['min_range'] / atr_perc

    df['trend_long'] = np.where(df['max_range'] > atr_perc, 1, 0)
    df['trend_short'] = np.where(df['min_range'] < -atr_perc, 1, 0)

    print(df['trend_long'].value_counts())
    print(df['trend_short'].value_counts())

    # Create a line chart
    fig = make_subplots(rows=1, cols=1, subplot_titles=('Price Range and ATR'))

    # Adding Max Range and Min Range to the plot
    fig.add_trace(go.Scatter(x=df.index, y=df['max_range'], mode='lines', name='Max Range'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['min_range'], mode='lines', name='Min Range'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=atr_perc * 1, mode='lines', name='+ATR (%)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=-atr_perc * 1, mode='lines', name='-ATR (%)'), row=1, col=1)

    # Update layout
    fig.update_layout(height=600, width=1200, title_text="Max/Min Price Range and ATR Over Time")
    fig.update_xaxes(title_text="Time", row=1, col=1)
    fig.update_yaxes(title_text="Percentage (%)", row=1, col=1)

    fig.show()

    # Distribution of max and min ranges
    # Creating subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Max Range Distribution', 'Min Range Distribution'))

    # Max Range Histogram
    fig.add_trace(go.Histogram(x=df['max_range'].dropna(), nbinsx=30, marker_color='blue'), row=1, col=1)

    # Min Range Histogram
    fig.add_trace(go.Histogram(x=df['min_range'].dropna(), nbinsx=30, marker_color='red'), row=1, col=2)

    # Update layout
    fig.update_layout(height=600, width=1200, title_text="Bitcoin Price Range Distributions")
    fig.update_xaxes(title_text="Max Range (%)", row=1, col=1)
    fig.update_xaxes(title_text="Min Range (%)", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)

    fig.show()

    display(pd.DataFrame({'max_range': df['max_range_atr'].describe().round(2), 'min_range': df['min_range_atr'].describe().round(2)}))

    return df


In [91]:
candles = download_1000_candles(market='ADA/USDT', tf='1h')
new_df = analyze_and_plot_bitcoin_ranges(candles)

trend_long
0    604
1    395
Name: count, dtype: int64
trend_short
1    608
0    391
Name: count, dtype: int64


,max_range,min_range
count,874.00,874.00
mean,1.61,-2.20
std,2.19,1.98
min,-3.78,-10.40
25%,0.26,-3.45
50%,0.85,-1.87
75%,2.23,-0.69
max,13.41,2.58


In [92]:
candles = download_1000_candles(market='BTC/USDT', tf='1h')
new_df = analyze_and_plot_bitcoin_ranges(candles)

trend_long
0    524
1    475
Name: count, dtype: int64
trend_short
1    556
0    443
Name: count, dtype: int64


,max_range,min_range
count,874.00,874.00
mean,2.07,-2.24
std,2.73,2.41
min,-5.02,-12.63
25%,0.44,-3.23
50%,1.14,-1.64
75%,2.39,-0.50
max,13.11,2.64


In [93]:
candles = download_1000_candles(market='BTC/USDT', tf='1h')
new_df = analyze_and_plot_bitcoin_ranges(candles, 12)

trend_long
0    648
1    351
Name: count, dtype: int64
trend_short
0    560
1    439
Name: count, dtype: int64


,max_range,min_range
count,887.00,887.00
mean,1.24,-1.38
std,1.86,1.77
min,-5.02,-10.99
25%,0.25,-2.05
50%,0.75,-0.98
75%,1.55,-0.26
max,11.44,2.71


In [94]:
candles = download_1000_candles(market='BTC/USDT', tf='1h')
new_df = analyze_and_plot_bitcoin_ranges(candles, 6)

trend_long
0    755
1    244
Name: count, dtype: int64
trend_short
0    699
1    300
Name: count, dtype: int64


,max_range,min_range
count,893.00,893.00
mean,0.74,-0.82
std,1.25,1.35
min,-5.02,-10.99
25%,0.06,-1.28
50%,0.50,-0.53
75%,1.08,-0.07
max,8.04,3.26
